In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# load data 

# House election Harvard Dataverse https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/IG0UN2
district_results = pd.read_csv('1976-2018-house.csv', header=0,encoding = 'unicode_escape')
district_results['candidatevotes'] = district_results['candidatevotes'].str.replace(',', '')
district_results['candidatevotes'] = district_results['candidatevotes'].astype(int)

# 2016 Election Results https://transition.fec.gov/general/FederalElections2016.shtml
pres_gen_results_2016 = pd.read_csv('2016 Pres General Results-Table 1.csv', header=0,encoding = 'unicode_escape')
sen_results_2016 = pd.read_csv('2016 US Senate Results by State-Table 1.csv', header=0,encoding = 'unicode_escape')
house_results_2016 = pd.read_csv('2016 US House Results by State-Table 1.csv', header=0,encoding = 'unicode_escape')

Election_2016 = district_results.loc[district_results['year'] == 2016]

sports_results = pd.read_csv('sports_data.csv')

# State
states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DC','DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

In [3]:
import glob

# get data file names
path =r'C:\DRO\DCL_rawdata_files'
filenames = glob.glob('District Data' + "/*.csv")


dfs = {}

filenames.sort()
i = 0 
for filename in filenames:
    dfs['state' + str(i)] = pd.read_csv(filename)
    i = i + 1
    

In [4]:
district_winners_2016 = pd.DataFrame()
for state in states : 
    temp = Election_2016.loc[Election_2016['state_po'] == state]
    #number of districts
    districts_number = temp['district'].max()
    while districts_number > 0 : 
        district = temp.loc[temp['district'] == districts_number]
        winner = pd.DataFrame(district.loc[district['candidatevotes'] == district['candidatevotes'].max()])
        district_winners_2016 = district_winners_2016.append(winner)
        districts_number = districts_number - 1

In [32]:
#will now correctly classify the winning party but doesn't put it into the original dataframe
    
for district in district_winners_2016.iterrows():
    # May be able to use the vote count to assign the correct value back to the original dataframe, 
    # only two winners shared an exact vote count so only 1 collision must be handled
    if district[1]['party'] == 'republican' : 
        district_winners_2016.loc[district[0],'class'] = 0
    elif district[1]['party'] == 'democrat' :
        district_winners_2016.loc[district[0],'class'] = 1
    else:
        district_winners_2016.loc[district[0],'class'] = 2

In [5]:

percentage_below_poverty_line = []
percent_bachelor = []
percent_hs = []
for state in dfs:
    num_cols = len(dfs[state].columns)
    stats_pl = dfs[state][dfs[state]['Subject'] == 'Percentage of Families and People Whose Income in the Past 12 Months is Below the Poverty Level'].iloc[0:1,]
    stats_ba = dfs[state][dfs[state]['Title'] == "Percent bachelor's degree or higher"]
    stats_hs = dfs[state][dfs[state]['Title'] == "Percent high school graduate or higher"]
    pl_mean_sum = 0
    hs_mean_sum = 0
    ba_mean_sum = 0
    for col in range(num_cols):
        if col < 3:
            continue
        elif (col % 2) != 0:
            pl_mean_sum += float(stats_pl.iloc[:,col].values[0])
            hs_mean_sum += float(stats_hs.iloc[:,col].values[0])
            ba_mean_sum += float(stats_ba.iloc[:,col].values[0])
        else:
            continue
            
    pl_mean = pl_mean_sum / ((num_cols - 3)/2)
    percentage_below_poverty_line.append(pl_mean)
    
    hs_mean = hs_mean_sum / ((num_cols - 3)/2)
    
    ba_mean = ba_mean_sum / ((num_cols - 3)/2)
    
    percent_hs.append(hs_mean)
    percent_bachelor.append(ba_mean)
    
    

In [6]:
sports_results['poverty'] = pd.Series(percentage_below_poverty_line)
sports_results['bachelors'] = pd.Series(percent_bachelor)
sports_results['hs'] = pd.Series(percent_hs)

In [64]:
#### LUKE
#Real GDP per capita by state by year
rGDP_pc_2015 = pd.read_csv('Real_GDP_pc/2015_rGDP_edit.csv')
rGDP_pc_2016 = pd.read_csv('Real_GDP_pc/2016_rGDP_edit.csv')
rGDP_pc_2017 = pd.read_csv('Real_GDP_pc/2017_rGDP_edit.csv')
#Unemployment delta by state, 2016--2018
## US national delta = -1%
unemp_delta = pd.read_csv('unemp_delta.csv')
rGDP_pc_2015.columns = ['state','2015_GDP']
economic_indicators = rGDP_pc_2015.copy()
economic_indicators['2016_GDP'] = rGDP_pc_2016['value']
economic_indicators['2017_GDP'] = rGDP_pc_2017['value']
economic_indicators = economic_indicators.merge(unemp_delta, on = 'state')

economic_indicators

,state,2015_GDP,2016_GDP,2017_GDP,difference
0,Alabama,39014,39201,39594,-1.9
1,Alaska,72943,71086,71274,-0.3
2,Arizona,41008,41643,42476,-0.6
3,Arkansas,38229,38303,38411,-0.3
4,California,62347,63785,66262,-1.3
5,Colorado,56708,57166,58686,0.1
6,Connecticut,67710,67845,68184,-1.0
7,Delaware,69976,66485,65554,-0.7
8,District of Columbia,173621,174150,173944,-0.5
9,Florida,41491,42013,42719,-1.2


In [38]:
sports_results

,State,cfb,nfl,nba,cbb,average,poverty,bachelors,hs
0,AK,0.5650,0.846,0.720,0.5170,0.662000,7.500000,30.200000,93.300000
1,AL,0.9330,0.813,0.293,0.5560,0.648750,12.471429,25.300000,86.485714
2,AR,0.1420,0.625,0.268,0.6570,0.423000,12.775000,23.100000,87.175000
3,AZ,0.6660,0.500,0.256,0.7710,0.548250,10.244444,29.400000,87.111111
4,CA,0.6360,0.813,0.707,0.6360,0.698000,9.303774,33.383019,83.313208
5,CO,0.4100,0.375,0.561,0.5310,0.469250,6.300000,41.471429,91.900000
6,CT,0.0830,0.688,0.671,0.4375,0.469875,7.080000,39.560000,90.940000
7,DC,0.5650,0.438,0.524,0.6010,0.532000,11.300000,60.400000,92.100000
8,DE,0.5830,0.563,0.634,0.4200,0.550000,8.400000,31.300000,89.800000
9,FL,0.7690,0.438,0.537,0.6170,0.590250,9.840741,30.314815,88.437037


In [52]:
district_winners_2016

,year,state,state_po,state_fips,state_cen,state_ic,office,district,stage,runoff,special,candidate,party,writein,mode,candidatevotes,totalvotes,unofficial,version,class
26853,2016,Alabama,AL,1,63,41,US House,7,gen,NaN,False,Terri A. Sewell,democrat,False,total,229330,233028.0,False,20171005,1.0
26851,2016,Alabama,AL,1,63,41,US House,6,gen,NaN,False,Gary J. Palmer,republican,False,total,245313,329306.0,False,20171005,0.0
26847,2016,Alabama,AL,1,63,41,US House,5,gen,NaN,False,Mo Brooks,republican,False,total,205647,308326.0,False,20171005,0.0
26846,2016,Alabama,AL,1,63,41,US House,4,gen,NaN,False,Robert B. Aderholt,republican,False,total,235925,239444.0,False,20171005,0.0
26843,2016,Alabama,AL,1,63,41,US House,3,gen,NaN,False,Mike Rogers,republican,False,total,192164,287104.0,False,20171005,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28256,2016,Wisconsin,WI,55,35,25,US House,5,gen,NaN,False,F. James Sensenbrenner Jr.,republican,False,total,260706,390844.0,False,20171005,0.0
28253,2016,Wisconsin,WI,55,35,25,US House,4,gen,NaN,False,Gwen Moore,democrat,False,total,220181,286909.0,False,20171005,1.0
28247,2016,Wisconsin,WI,55,35,25,US House,3,gen,NaN,False,Ron Kind,democrat,False,total,257401,260370.0,False,20171005,1.0
28244,2016,Wisconsin,WI,55,35,25,US House,2,gen,NaN,False,Mark Pocan,democrat,False,total,273537,398060.0,False,20171005,1.0


In [70]:
model_frame = district_winners_2016[['state_po','state_fips','district', 'class']]

In [73]:
model_frame = model_frame.join(sports_results.set_index('State'), on='state_po', how='inner')

In [76]:
model_frame = model_frame.drop('state_fips', axis=1)

In [78]:
model_frame['state_district'] = model_frame.apply(lambda row: row.state_po + str(row.district), axis=1)

In [83]:
model_frame = model_frame.drop(['average','district'], axis=1)

In [86]:
model_frame.set_index('state_district')

,state_po,class,cfb,nfl,nba,cbb,poverty,bachelors,hs
state_district,,,,,,,,,
AL7,AL,1.0,0.933,0.813,0.293,0.556,12.471429,25.3,86.485714
AL6,AL,0.0,0.933,0.813,0.293,0.556,12.471429,25.3,86.485714
AL5,AL,0.0,0.933,0.813,0.293,0.556,12.471429,25.3,86.485714
AL4,AL,0.0,0.933,0.813,0.293,0.556,12.471429,25.3,86.485714
AL3,AL,0.0,0.933,0.813,0.293,0.556,12.471429,25.3,86.485714
...,...,...,...,...,...,...,...,...,...
WI5,WI,0.0,0.615,0.406,0.537,0.450,7.162500,29.9,92.037500
WI4,WI,1.0,0.615,0.406,0.537,0.450,7.162500,29.9,92.037500
WI3,WI,1.0,0.615,0.406,0.537,0.450,7.162500,29.9,92.037500
